<a href="https://colab.research.google.com/github/tomifarall/Orga-Datos-Tp2/blob/master/modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud
import re
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances

import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
#from catboost import CatBoostRegressor
#!pip install catboost

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
#ESTO NO LO EJECUTEN SI UTILIZAN UNA VERSION DE PC , YO LO HAGO PARA PODER USAR EL TRAIN DE MI DRIVE,Y USAR EL GITHUB ONLINE
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1RJH-BaqHHtZzrSgu6yUVc8bRXcfev9tB'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('train_1.csv')

In [4]:
# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1sivn_RJ13uL13DmmB_G03yJRJWw_b2He'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('train_2.csv')

In [5]:
# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1eO4Vnym1_XoOx8yL1UC9WmZJvROH4dt_'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('train_3.csv')

In [89]:
train_1 = pd.read_csv('train_1.csv')
train_2= pd.read_csv('train_2.csv')
train_3=pd.read_csv('train_3.csv')

In [59]:
train_3.head(10)

,Unnamed: 0,id,keyword,location,text,target,clean_text
0,0,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,bbcmtd wholesale market ablaze
1,1,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,always try bring heavy metal rt
2,2,50,ablaze,Africa,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,africanbaze breaking newsnigeria flag set abla...
3,3,52,ablaze,Philadelphia,Crying out for more! Set me ablaze,0,cry set ablaze
4,4,53,ablaze,Florida,On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,plus side look sky last night ablaze
5,5,54,ablaze,Pretoria,@PhDSquares #mufc they've built so much hype a...,0,phdsquares mufc theyve built much hype around ...
6,6,55,ablaze,World Wide!!,INEC Office in Abia Set Ablaze - http://t.co/3...,1,inec office abia set ablaze
7,7,57,ablaze,Paranaque City,Ablaze for you Lord :D,0,ablaze lord
8,8,59,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http:/...,0,check nsfw
9,9,62,ablaze,milky way,Had an awesome time visiting the CFC head offi...,0,awesome time visiting cfc head office ancop si...


In [90]:
train_3=train_3.drop( columns='Unnamed: 0')

In [9]:
!pip install gensim
from gensim.models import KeyedVectors

In [10]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2020-07-29 12:35:56--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.39.70
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.39.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  66.1MB/s    in 34s     

2020-07-29 12:36:30 (45.9 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [11]:
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz' # from above
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [88]:
def agregarfeature(archivo):

  
  #agrego longitud
  archivo['length']=archivo['text'].str.len()
  #vectorizo el text
  #vectorizer= CountVectorizer(max_features=40)
  #features=vectorizer.fit_transform(archivo['text']).todense()
  #txt_encode_frame=pd.DataFrame(features)
  #archivo[txt_encode_frame.columns]=txt_encode_frame

 
  
  archivo.drop(['text','id'], axis=1, inplace=True)
  
  #categorizo los null de keyword y location
  #archivo['keyword'].fillna('no keyword', inplace = True) 
  #archivo['location'].fillna('no location', inplace = True)

  #agrego longitud keyword y elimino keyword(despues tendria que mejorar eso)
  archivo['keyword'] = archivo['keyword'].str.replace('%20', ' ')
  archivo['keyword_length'] = archivo['keyword'].str.len()
  archivo.drop(['keyword'], axis=1, inplace=True)

   

  #limito los paises , solo aptos la cantidad de tweet mayor a 20


  #overtwentytweet = archivo.loc[:].groupby('location').agg({'location':'count'}).nlargest(10,'location')
  #overtwentytweet.columns=['canttweet']
  #dataovertwentytweet=overtwentytweet.reset_index()
  #dataovertwentytweet=dataovertwentytweet[dataovertwentytweet['canttweet']>15]
  #archivo=pd.merge(archivo,dataovertwentytweet,on='location',how='inner')

  #mean enconding location

 # archivo=archivo.groupby('location').agg(mean_encoding_location=('target','mean')).reset_index()
  #archivo.drop(['location'], axis=1, inplace=True)
  print('... sucessfull')

In [91]:
agregarfeature(train_3)

... sucessfull


In [92]:
def document_vector(word2vec_model, doc):
    acum_doc=list()
    for word in doc:
        if word in word2vec_model.vocab:
          acum_doc.append(word)
    return np.mean(word2vec_model[acum_doc], axis=0) 

In [93]:
train_3['text_embbeding_prom']=train_3['clean_text'].apply(lambda x: document_vector(word2vec,x))

In [94]:
for lista in train_3['text_embbeding_prom']:
    cont=0
    for a in lista:
      cont=cont +1
      train_3[cont]=a

In [95]:
train_3.head(1)

,location,target,clean_text,length,keyword_length,text_embbeding_prom,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,...,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300
0,Birmingham,1,bbcmtd wholesale market ablaze,55,6.0,"[-0.15567149, 0.13058339, -0.022577701, 0.1368...",-0.172481,0.102126,0.011953,0.146214,-0.052405,0.035479,-0.086294,-0.051557,-0.047721,0.014978,-0.040102,-0.109308,-0.230012,0.031233,-0.120964,0.09409,0.112859,0.158377,-0.005189,0.031278,-0.238707,-0.029835,0.123439,0.024576,-0.092884,0.05367,-0.249286,0.045073,-0.017839,-0.024635,-0.001338,0.026476,-0.072191,-0.101755,...,0.068903,-0.016415,0.091724,-0.019002,-0.096512,-0.00142,-0.009515,-0.000695,-0.030187,-0.06625,-0.073244,-0.036969,0.024197,0.052804,0.069427,-0.086794,-0.160874,-0.133805,-0.100515,0.061858,-0.08632,0.13997,0.011669,0.006944,-0.003619,0.069872,0.003962,-0.027198,-0.064835,-0.030085,0.070977,-0.007601,-0.092008,0.075024,-0.030936,-0.144448,-0.098428,-0.026763,-0.127965,0.159836


In [99]:
#mean_enconding
train_3['mean_encoding_location']=train_3.groupby('location').agg(mean_encoding_location=('target','mean')).reset_index()['mean_encoding_location']
train_3

,location,target,clean_text,length,keyword_length,text_embbeding_prom,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,...,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,mean_encoding_location
0,Birmingham,1,bbcmtd wholesale market ablaze,55,6.0,"[-0.15567149, 0.13058339, -0.022577701, 0.1368...",-0.172481,0.102126,0.011953,0.146214,-0.052405,0.035479,-0.086294,-0.051557,-0.047721,0.014978,-0.040102,-0.109308,-0.230012,0.031233,-0.120964,0.09409,0.112859,0.158377,-0.005189,0.031278,-0.238707,-0.029835,0.123439,0.024576,-0.092884,0.05367,-0.249286,0.045073,-0.017839,-0.024635,-0.001338,0.026476,-0.072191,-0.101755,...,-0.016415,0.091724,-0.019002,-0.096512,-0.00142,-0.009515,-0.000695,-0.030187,-0.06625,-0.073244,-0.036969,0.024197,0.052804,0.069427,-0.086794,-0.160874,-0.133805,-0.100515,0.061858,-0.08632,0.13997,0.011669,0.006944,-0.003619,0.069872,0.003962,-0.027198,-0.064835,-0.030085,0.070977,-0.007601,-0.092008,0.075024,-0.030936,-0.144448,-0.098428,-0.026763,-0.127965,0.159836,1.0
1,Est. September 2012 - Bristol,0,always try bring heavy metal rt,67,6.0,"[-0.18583818, 0.1286427, -0.020851828, 0.12635...",-0.172481,0.102126,0.011953,0.146214,-0.052405,0.035479,-0.086294,-0.051557,-0.047721,0.014978,-0.040102,-0.109308,-0.230012,0.031233,-0.120964,0.09409,0.112859,0.158377,-0.005189,0.031278,-0.238707,-0.029835,0.123439,0.024576,-0.092884,0.05367,-0.249286,0.045073,-0.017839,-0.024635,-0.001338,0.026476,-0.072191,-0.101755,...,-0.016415,0.091724,-0.019002,-0.096512,-0.00142,-0.009515,-0.000695,-0.030187,-0.06625,-0.073244,-0.036969,0.024197,0.052804,0.069427,-0.086794,-0.160874,-0.133805,-0.100515,0.061858,-0.08632,0.13997,0.011669,0.006944,-0.003619,0.069872,0.003962,-0.027198,-0.064835,-0.030085,0.070977,-0.007601,-0.092008,0.075024,-0.030936,-0.144448,-0.098428,-0.026763,-0.127965,0.159836,0.0
2,Africa,1,africanbaze breaking newsnigeria flag set abla...,82,6.0,"[-0.1737264, 0.0682763, 0.02677748, 0.16577148...",-0.172481,0.102126,0.011953,0.146214,-0.052405,0.035479,-0.086294,-0.051557,-0.047721,0.014978,-0.040102,-0.109308,-0.230012,0.031233,-0.120964,0.09409,0.112859,0.158377,-0.005189,0.031278,-0.238707,-0.029835,0.123439,0.024576,-0.092884,0.05367,-0.249286,0.045073,-0.017839,-0.024635,-0.001338,0.026476,-0.072191,-0.101755,...,-0.016415,0.091724,-0.019002,-0.096512,-0.00142,-0.009515,-0.000695,-0.030187,-0.06625,-0.073244,-0.036969,0.024197,0.052804,0.069427,-0.086794,-0.160874,-0.133805,-0.100515,0.061858,-0.08632,0.13997,0.011669,0.006944,-0.003619,0.069872,0.003962,-0.027198,-0.064835,-0.030085,0.070977,-0.007601,-0.092008,0.075024,-0.030936,-0.144448,-0.098428,-0.026763,-0.127965,0.159836,1.0
3,Philadelphia,0,cry set ablaze,34,6.0,"[-0.17176513, 0.11408691, -0.0016967773, 0.137...",-0.172481,0.102126,0.011953,0.146214,-0.052405,0.035479,-0.086294,-0.051557,-0.047721,0.014978,-0.040102,-0.109308,-0.230012,0.031233,-0.120964,0.09409,0.112859,0.158377,-0.005189,0.031278,-0.238707,-0.029835,0.123439,0.024576,-0.092884,0.05367,-0.249286,0.045073,-0.017839,-0.024635,-0.001338,0.026476,-0.072191,-0.101755,...,-0.016415,0.091724,-0.019002,-0.096512,-0.00142,-0.009515,-0.000695,-0.030187,-0.06625,-0.073244,-0.036969,0.024197,0.052804,0.069427,-0.086794,-0.160874,-0.133805,-0.100515,0.061858,-0.08632,0.13997,0.011669,0.006944,-0.003619,0.069872,0.003962,-0.027198,-0.064835,-0.030085,0.070977,-0.007601,-0.092008,0.075024,-0.030936,-0.144448,-0.098428,-0.026763,-0.127965,0.159836,0.0
4,Florida,0,plus side look sky last night ablaze,76,6.0,"[-0.15891972, 0.10336869, 0.020589193, 0.13127...",-0.172481,0.102126,0.011953,0.146214,-0.052405,0.035479,-0.086294,-0.051557,-0.047721,0.014978,-0.040102,-0.109308,-0.230012,0.031233,-0.120964,0.09409,0.112859,0.158377,-0.005189,0.031278,-0.238707,-0.029835,0.123439,0.024576,-0.092884,0.05367,-0.249286,0.045073,-0.017839,-0.024635,-0.00

In [100]:
train_3.drop(['location'], axis=1, inplace=True)

In [101]:
train_3.drop(['clean_text'], axis=1, inplace=True)

In [102]:
train_3.drop(['text_embbeding_prom'], axis=1, inplace=True) 	

In [103]:
train_3.head(2)

,target,length,keyword_length,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,mean_encoding_location
0,1,55,6.0,-0.172481,0.102126,0.011953,0.146214,-0.052405,0.035479,-0.086294,-0.051557,-0.047721,0.014978,-0.040102,-0.109308,-0.230012,0.031233,-0.120964,0.09409,0.112859,0.158377,-0.005189,0.031278,-0.238707,-0.029835,0.123439,0.024576,-0.092884,0.05367,-0.249286,0.045073,-0.017839,-0.024635,-0.001338,0.026476,-0.072191,-0.101755,-0.144031,0.085957,-0.214241,...,-0.016415,0.091724,-0.019002,-0.096512,-0.00142,-0.009515,-0.000695,-0.030187,-0.06625,-0.073244,-0.036969,0.024197,0.052804,0.069427,-0.086794,-0.160874,-0.133805,-0.100515,0.061858,-0.08632,0.13997,0.011669,0.006944,-0.003619,0.069872,0.003962,-0.027198,-0.064835,-0.030085,0.070977,-0.007601,-0.092008,0.075024,-0.030936,-0.144448,-0.098428,-0.026763,-0.127965,0.159836,1.0
1,0,67,6.0,-0.172481,0.102126,0.011953,0.146214,-0.052405,0.035479,-0.086294,-0.051557,-0.047721,0.014978,-0.040102,-0.109308,-0.230012,0.031233,-0.120964,0.09409,0.112859,0.158377,-0.005189,0.031278,-0.238707,-0.029835,0.123439,0.024576,-0.092884,0.05367,-0.249286,0.045073,-0.017839,-0.024635,-0.001338,0.026476,-0.072191,-0.101755,-0.144031,0.085957,-0.214241,...,-0.016415,0.091724,-0.019002,-0.096512,-0.00142,-0.009515,-0.000695,-0.030187,-0.06625,-0.073244,-0.036969,0.024197,0.052804,0.069427,-0.086794,-0.160874,-0.133805,-0.100515,0.061858,-0.08632,0.13997,0.011669,0.006944,-0.003619,0.069872,0.003962,-0.027198,-0.064835,-0.030085,0.070977,-0.007601,-0.092008,0.075024,-0.030936,-0.144448,-0.098428,-0.026763,-0.127965,0.159836,0.0


In [104]:
target=train_3['target']

In [105]:
train_3.drop(['target'], axis=1, inplace=True)


In [106]:
train_3.head(1)

,length,keyword_length,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,mean_encoding_location
0,55,6.0,-0.172481,0.102126,0.011953,0.146214,-0.052405,0.035479,-0.086294,-0.051557,-0.047721,0.014978,-0.040102,-0.109308,-0.230012,0.031233,-0.120964,0.09409,0.112859,0.158377,-0.005189,0.031278,-0.238707,-0.029835,0.123439,0.024576,-0.092884,0.05367,-0.249286,0.045073,-0.017839,-0.024635,-0.001338,0.026476,-0.072191,-0.101755,-0.144031,0.085957,-0.214241,0.098976,...,-0.016415,0.091724,-0.019002,-0.096512,-0.00142,-0.009515,-0.000695,-0.030187,-0.06625,-0.073244,-0.036969,0.024197,0.052804,0.069427,-0.086794,-0.160874,-0.133805,-0.100515,0.061858,-0.08632,0.13997,0.011669,0.006944,-0.003619,0.069872,0.003962,-0.027198,-0.064835,-0.030085,0.070977,-0.007601,-0.092008,0.075024,-0.030936,-0.144448,-0.098428,-0.026763,-0.127965,0.159836,1.0


In [107]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
train_test_split(train_3, target, test_size=0.2, random_state=123)

In [ ]:
#xgboost colsample_bytree = 0.3, learning_rate = 0.01,
              #  max_depth = 4, alpha = 10, n_estimators = 10) 4.9 , sin embeding

In [140]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', 
                colsample_bytree = 0.3, learning_rate = 0.001,
                max_depth = 4, alpha = 10, n_estimators = 3)

In [141]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0,
             importance_type='gain', learning_rate=0.001, max_delta_step=0,
             max_depth=4, min_child_weight=1, missing=None, n_estimators=3,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [142]:
preds = xg_reg.predict(X_test)

In [143]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.499952
